In [1]:
import os
import time
import argparse
import math
from numpy import finfo
import IPython.display as ipd
import torch

In [2]:

from distributed import apply_gradient_allreduce
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

from model import Tacotron2
from data_utils import TextMelLoader, TextMelCollate
from loss_function import Tacotron2Loss
from logger import Tacotron2Logger
from hparams import create_hparams

In [3]:
hparams = create_hparams()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [11]:
from text import text_to_sequence, cmudict, candict


In [ ]:
text_to_sequence("wai4-juk1-sou3-sou3-hou2-hoi1-sam1-gam3-waa6", hparams.text_cleaners, )

In [4]:
trainset = TextMelLoader(hparams.training_files, hparams)

In [7]:
for each_set in trainset:
    pass

In [ ]:
batch = [trainset[1], trainset[0]]

In [ ]:
input_lengths, ids_sorted_decreasing = torch.sort(
            torch.LongTensor([len(x[0]) for x in batch]),
            dim=0, descending=True)
max_input_len = input_lengths[0]
text_padded = torch.LongTensor(len(batch), max_input_len)
text_padded.zero_()
for i in range(len(ids_sorted_decreasing)):
    text = batch[ids_sorted_decreasing[i]][0]
    text_padded[i, :text.size(0)] = text

# Right zero-pad mel-spec
num_mels = batch[0][1].size(0)
max_target_len = max([x[1].size(1) for x in batch])


# include mel padded and gate padded
mel_padded = torch.FloatTensor(len(batch), num_mels, max_target_len)
mel_padded.zero_()
gate_padded = torch.FloatTensor(len(batch), max_target_len)
gate_padded.zero_()
output_lengths = torch.LongTensor(len(batch))
spk_ids = torch.LongTensor([batch[i][2] for i in range(len(ids_sorted_decreasing))])
for i in range(len(ids_sorted_decreasing)):
    mel = batch[ids_sorted_decreasing[i]][1]
    mel_padded[i, :, :mel.size(1)] = mel
    gate_padded[i, mel.size(1) - 1:] = 1
    output_lengths[i] = mel.size(1)

In [ ]:
text_padded

In [ ]:
spk_ids

In [ ]:
trainset[1]

In [ ]:
trainset[0]